<a href="https://colab.research.google.com/github/tjweir/hackfromhome2021-wsid/blob/main/Simulate_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Data Simulation

In [ ]:
num_houses = 20000

# define energy usage, this could be converted to KWH or something reasonable
#energy_usage = [500,2000] # range of energy usage, reasonable scale??

# define features
# ------------------------------------------------------------------------------
# define categories
energy_types = ['grid','solar']
heating_types = ['heat pump', 'natural gas', 'electric']
glazing_types = ['single','double','triple']
vehicle_types = ['electric','general']
domicile_types = ['detached','semi detached','terraced'] 
season = ['Summer']
Temp_cat = ['High', 'Medium', 'Low']
Housesize = ['Small', 'Medium', 'Large'] #Catogories based on surface area
insulation = ['High', 'Medium', 'Low']
SmartHouse = ['Semi', 'Full']
EPC_rating = ['A','B','C','D','E','F','G'] 



# define ranges
num_occupants = [1,6] # range of occupants
year_built = [1930,2000] # range of years built
num_appliances = [10,50] # range of appliances
insulation_range = [1.5,7] # r value/ something to be review
latitude = [50.1, 59.9]
longitude = [-7.64, 1.65]

temperature = [16,25] # potentially categories: high, medium, low
num_floors = [1,3]
surface_area = [100,500] # square meter

# simulate data, random selection of the above features
# ------------------------------------------------------------------------------
df = pd.DataFrame(np.arange(num_houses), columns=['House'])

# categories

df['latitude'] = np.random.default_rng().uniform(low=latitude[0], high=latitude[1], size=num_houses)
df['longitude'] = np.random.default_rng().uniform(low=longitude[0], high=longitude[1], size=num_houses)

df['energy_type'] = np.random.choice(energy_types, size=num_houses)
  
df['heating_type'] = np.where(df['energy_type'] == 'solar', 'electric', np.random.choice(heating_types, size=num_houses))

df['glazing_types'] = np.random.choice(glazing_types, size=num_houses)

df['vehicle_type'] = np.random.choice(vehicle_types, size=num_houses)

df['domicile_type'] = np.random.choice(domicile_types, size=num_houses)

df['season'] = np.random.choice(season, size=num_houses)

df['temperature'] = np.random.randint(low=temperature[0], high=temperature[1], size=num_houses)
def temp(df):
  if (df['temperature'] < 20):
    return 'Low'
  elif (df['temperature'] > 20 & df['temperature'] <= 23 ):
    return 'Medium'
  else: return 'High'
df['Temp_cat'] = df.apply(temp, axis=1)

df['house_size'] = np.random.randint(low=surface_area[0], high=surface_area[1], size=num_houses)
def surface(df):
  if (df['house_size'] < 200) :
    return 'Small'
  elif (df['house_size'] > 200 & df['house_size'] <= 300 ):
    return 'Medium'
  else: return 'Large'
df['house_size'] = df.apply(surface, axis=1)



df['num_occupants'] = np.random.randint(low=num_occupants[0], high=num_occupants[1], size=num_houses)
df['year_built'] = np.random.randint(low=year_built[0], high=year_built[1], size=num_houses)


df['insulation'] = np.random.choice(insulation, size=len(df))
def insulation(df):
  if (df['glazing_types'] == 'single') :
    return 'Low'
  elif (df['glazing_types'] == "double"):
    return 'Medium'
  else: return 'High'
df['insulation'] = df.apply(insulation, axis=1)


df['SmartHouse'] = np.random.choice(SmartHouse, size=len(df))

df['EPC_rating'] = np.random.choice(EPC_rating, size=len(df))
def EPC_rating(df):
  if (df['glazing_types'] == 'single') :
    return np.random.choice(['F', 'G'])
  elif (df['glazing_types'] == "double"):
    return np.random.choice(['C', 'D', 'E'])
  else: return np.random.choice(['A', 'B'])
df['EPC_rating'] = df.apply(EPC_rating, axis=1)


df['num_appliances'] = np.where(df['house_size'] == 'Small',  np.random.randint(low=10, high=20, size=num_houses),  np.random.randint(low=10, high=50, size=num_houses))

# energy usage - per month
def eupm(df):
    if df['house_size'] == 'Small':
      return np.random.randint(low=500, high=1000)
    elif df['house_size'] == 'Medium':
      return np.random.randint(low=700, high=1500)
    elif df['house_size'] == 'Large':
      return np.random.randint(low=1000, high=3000)


df['energy_usage_pm'] = df.apply(eupm, axis=1)

In [ ]:
df

,House,latitude,longitude,energy_type,heating_type,glazing_types,vehicle_type,domicile_type,season,temperature,Temp_cat,house_size,num_occupants,year_built,insulation,SmartHouse,EPC_rating,num_appliances,energy_usage_pm
0,0,50.123504,-6.616434,solar,electric,double,electric,terraced,Summer,17,Low,Medium,5,1970,Medium,Semi,C,20,753
1,1,56.708603,-0.457610,solar,electric,triple,general,detached,Summer,18,Low,Medium,3,1995,High,Semi,B,32,1023
2,2,59.486333,-0.625680,solar,electric,double,electric,semi detached,Summer,23,Medium,Medium,3,1967,Medium,Semi,E,48,872
3,3,55.997358,-0.498099,solar,electric,single,general,detached,Summer,18,Low,Medium,4,1988,Low,Full,F,38,1447
4,4,56.043025,-4.002039,solar,electric,double,general,semi detached,Summer,23,Medium,Medium,4,1970,Medium,Full,E,37,1023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,50.232064,-0.041656,solar,electric,double,electric,terraced,Summer,24,Medium,Medium,5,1933,Medium,Semi,D,32,1450
19996,19996,54.620908,0.682945,solar,electric,double,general,detached,Summer,19,Low,Medium,1,1960,Medium,Full,E,42,964
19997,19997,52.016146,-1.274549,solar,electric,double,general,detached,Summer,16,Low,Medium,5,1967,Medium,Semi,C,27,731
19998,19998,53.741860,-2.846520,solar,electric,triple,general,detached,Summer,20,High,Medium,3,1963,High,Full,A,11,802


# Giving Features some correlation

In [ ]:
# define the relationship between features and energy_usage.
# relationship is indicated in a range of percentages.
# this ensures that if we change the scale of energy_usage (e.g. to KWH), we keep the same relationships
# the actual percentage is randomly chosen for each house within the given range

# dict within dict.
# 2nd level dict is always categorical.
delta_energy = {'energy_type': {'grid':[0,0], 'solar':[-25,-20]}, # step difference in usage with solar
                'heating_type': {'heat pump': [-20,-10], 'natural gas':[0,0], 'electric':[+8,+12]}, # step difference in usage
                'vehicle_type': {'electric':[+10,+20], 'general':[0,0]}, # more usage with electric vehicle
                'domicile_type': {'detached':[+10, +20],'semi detached':[0,0],'terraced':[-10,-15]}, # usage difference with domicile
                'occupants':[+3,+8], # extra usage per occupant
                'year_built':[-1,-3], # extra (less) usage per year
                'num_appliances':[+1,+3], # extra usage per appliance
                
                'insulation':[-2,-5], # less with each increase in r
                'EPC_rating': {'A': [-20,-15], 'B':[-10, -5], 'C':[+1,+5], 'D': [+10,+15], 'E' : [+15,+20], 'F' : [+25,+30], "G": [+35,+50]}
                #'temperature_winter':[+3,+6], # more with each increase in degrees
                #'temperature_summer':[-3,-6], # less with each increase in degrees
                #'num_floors':[+3,+8], # more with more floors
                #'surface_area':[+0.2,+0.5] # more with more surface area
                }


# loop over dicts in delta_energy
for key_outer, ftypes_outer in delta_energy.items():

  # if 2nd level is also dict, the types are categorical. 
  # In this case, key_outer is the column name (category type) and key_inner is the category name
  if isinstance(ftypes_outer, dict):
    for key_inner, ftypes_inner in ftypes_outer.items():
      
      # select the entries that match the category name
      idx = df.loc[:,key_outer]==key_inner
      n = np.sum(idx)

      # select the random change (within the range) to the energy_usage
      random_change = np.random.default_rng().uniform(low=ftypes_inner[0], high=ftypes_inner[1], size=n)
      random_change = (100 + random_change)/100 # convert to proportion

      # replace energy_usage
      df['energy_usage_pm'][idx] *= random_change
      # df.iloc[idx,'energy_usage'] *= random_change  

  elif isinstance(ftypes_outer, list):

    # select the random change (within the range) to the energy_usage
    random_change = np.random.default_rng().uniform(low=ftypes_outer[0], high=ftypes_outer[1], size=num_houses)
    random_change = (100 + random_change)/100 # convert to proportion

    # replace energy_usage
    df.energy_usage_pm *= random_change



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df

,House,latitude,longitude,energy_type,heating_type,glazing_types,vehicle_type,domicile_type,season,temperature,Temp_cat,house_size,num_occupants,year_built,insulation,SmartHouse,EPC_rating,num_appliances,energy_usage_pm
0,0,50.123504,-6.616434,solar,electric,double,electric,terraced,Summer,17,Low,Medium,5,1970,Medium,Semi,C,20,737.209626
1,1,56.708603,-0.457610,solar,electric,triple,general,detached,Summer,18,Low,Medium,3,1995,High,Semi,B,32,909.524100
2,2,59.486333,-0.625680,solar,electric,double,electric,semi detached,Summer,23,Medium,Medium,3,1967,Medium,Semi,E,48,957.015634
3,3,55.997358,-0.498099,solar,electric,single,general,detached,Summer,18,Low,Medium,4,1988,Low,Full,F,38,1838.451339
4,4,56.043025,-4.002039,solar,electric,double,general,semi detached,Summer,23,Medium,Medium,4,1970,Medium,Full,E,37,1101.326663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,50.232064,-0.041656,solar,electric,double,electric,terraced,Summer,24,Medium,Medium,5,1933,Medium,Semi,D,32,1397.153679
19996,19996,54.620908,0.682945,solar,electric,double,general,detached,Summer,19,Low,Medium,1,1960,Medium,Full,E,42,1177.725917
19997,19997,52.016146,-1.274549,solar,electric,double,general,detached,Summer,16,Low,Medium,5,1967,Medium,Semi,C,27,771.964969
19998,19998,53.741860,-2.846520,solar,electric,triple,general,detached,Summer,20,High,Medium,3,1963,High,Full,A,11,622.080005


In [ ]:
df2 = df.copy()

# Following the same procedure for "winter" season

In [ ]:
df2['season'] = 'Winter'

In [ ]:
def eupm(df):
    if df['house_size'] == 'Small':
      return np.random.randint(low=700, high=1100)
    elif df['house_size'] == 'Medium':
      return np.random.randint(low=1000, high=1700)
    else: return np.random.randint(low=1500, high=3500)


df2['energy_usage_pm'] = df.apply(eupm, axis=1)

In [ ]:
delta_energy = {'energy_type': {'grid':[0,0], 'solar':[-25,-20]}, # step difference in usage with solar
                'heating_type': {'heat pump': [-20,-10], 'natural gas':[0,0], 'electric':[+8,+12]}, # step difference in usage
                'vehicle_type': {'electric':[+10,+20], 'general':[0,0]}, # more usage with electric vehicle
                'domicile_type': {'detached':[+10, +20],'semi detached':[0,0],'terraced':[-10,-15]}, # usage difference with domicile
                'occupants':[+3,+8], # extra usage per occupant
                'year_built':[-1,-3], # extra (less) usage per year
                'num_appliances':[+1,+3], # extra usage per appliance
                'insulation':[-2,-5], # less with each increase in r
                'EPC_rating': {'A': [-20,-15], 'B':[-10, -5], 'C':[+1,+5], 'D': [+10,+15], 'E' : [+15,+20], 'F' : [+25,+30], "G": [+35,+50]}
                #'temperature_winter':[+3,+6], # more with each increase in degrees
                #'temperature_summer':[-3,-6], # less with each increase in degrees
                #'num_floors':[+3,+8], # more with more floors
                #'surface_area':[+0.2,+0.5] # more with more surface area
                }


# loop over dicts in delta_energy
for key_outer, ftypes_outer in delta_energy.items():

  # if 2nd level is also dict, the types are categorical. 
  # In this case, key_outer is the column name (category type) and key_inner is the category name
  if isinstance(ftypes_outer, dict):
    for key_inner, ftypes_inner in ftypes_outer.items():
      
      # select the entries that match the category name
      idx = df2.loc[:,key_outer]==key_inner
      n = np.sum(idx)

      # select the random change (within the range) to the energy_usage
      random_change = np.random.default_rng().uniform(low=ftypes_inner[0], high=ftypes_inner[1], size=n)
      random_change = (100 + random_change)/100 # convert to proportion

      # replace energy_usage
      df2['energy_usage_pm'][idx] *= random_change
      # df.iloc[idx,'energy_usage'] *= random_change  

  elif isinstance(ftypes_outer, list):

    # select the random change (within the range) to the energy_usage
    random_change = np.random.default_rng().uniform(low=ftypes_outer[0], high=ftypes_outer[1], size=num_houses)
    random_change = (100 + random_change)/100 # convert to proportion

    # replace energy_usage
    df2.energy_usage_pm *= random_change

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df_new = pd.concat([df,df2])

In [ ]:
df_new

,House,latitude,longitude,energy_type,heating_type,glazing_types,vehicle_type,domicile_type,season,temperature,Temp_cat,house_size,num_occupants,year_built,insulation,SmartHouse,EPC_rating,num_appliances,energy_usage_pm
0,0,50.123504,-6.616434,solar,electric,double,electric,terraced,Summer,17,Low,Medium,5,1970,Medium,Semi,C,20,737.209626
1,1,56.708603,-0.457610,solar,electric,triple,general,detached,Summer,18,Low,Medium,3,1995,High,Semi,B,32,909.524100
2,2,59.486333,-0.625680,solar,electric,double,electric,semi detached,Summer,23,Medium,Medium,3,1967,Medium,Semi,E,48,957.015634
3,3,55.997358,-0.498099,solar,electric,single,general,detached,Summer,18,Low,Medium,4,1988,Low,Full,F,38,1838.451339
4,4,56.043025,-4.002039,solar,electric,double,general,semi detached,Summer,23,Medium,Medium,4,1970,Medium,Full,E,37,1101.326663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,50.232064,-0.041656,solar,electric,double,electric,terraced,Winter,24,Medium,Medium,5,1933,Medium,Semi,D,32,1662.740557
19996,19996,54.620908,0.682945,solar,electric,double,general,detached,Winter,19,Low,Medium,1,1960,Medium,Full,E,42,1514.412651
19997,19997,52.016146,-1.274549,solar,electric,double,general,detached,Winter,16,Low,Medium,5,1967,Medium,Semi,C,27,1328.893336
19998,19998,53.741860,-2.846520,solar,electric,triple,general,detached,Winter,20,High,Medium,3,1963,High,Full,A,11,1364.354161


In [ ]:
df_new.to_excel("/content/drive/MyDrive/Hackathon/FinalData_20000.xlsx", index=False)